# Cài đặt thư viện

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

# Kết nối Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
folder = "/content/drive/MyDrive/IE224-DataAnalysis"

# Import thư viện

In [ ]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

# Cài đặt Selenium

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

# Crawl Data

## Lấy danh sách link

In [ ]:
element_class = "AdParam_adParamValue__IfaYa"
tag_a_class = "AdItem_adItem__gDDQT"

In [ ]:
tags_a_pages = []
file_path = f"{folder}/links_200.txt"
file = open(file_path, 'w')

for i in range(1, 201):
  driver.get(f"https://www.chotot.com/mua-ban-laptop?page={i}")
  tags = driver.find_elements(By.CSS_SELECTOR, f"a.{tag_a_class}")
  for tag in tags:
    file.write(tag.get_attribute('href').split('#')[0])
    file.write('\n')

file.close()

In [ ]:
driver.quit()

## Lấy dữ liệu trong từng link

In [ ]:
data = []

with open(f'{folder}/links_200.txt', 'r') as f:
  links = f.read().split()

for link in tqdm(links, desc="Processing"):
  html = requests.get(link).text
  soup = BeautifulSoup(html, 'html.parser')
  aditems = soup.find_all(class_="AdParam_adParamValue__IfaYa")
  if len(aditems):
    data.append({
        item.find_previous_sibling("span").text.split(':')[0] : item.text
        for item in aditems
    })
    price = soup.find(class_="AdDecription_price__L2jjH").find("span").text
    data[-1]['Giá'] = price

Processing: 100%|██████████| 4010/4010 [1:34:48<00:00,  1.42s/it]


In [ ]:
csv_file = f"{folder}/data_laptop_price_200.csv"
headers = ['Tình trạng', 'Hãng', 'Dòng máy', 'Bộ vi xử lý', 'RAM', 'Ổ cứng', 'Thông tin sử dụng',
           'Card màn hình', 'Kích cỡ màn hình', 'Chính sách bảo hành', 'Xuất xứ','classify', "Giá"]

with open(csv_file, 'w', newline='') as file:
  writer = csv.DictWriter(file, fieldnames=headers)

  writer.writeheader()

  for value in data:
    row = {}
    for key in headers:
        row[key] = value.get(key, None)
    row["classify"] = 'laptop'

    writer.writerow(row)

In [ ]:
columns=['elt_condition', 'pc_brand', 'pc_model','elt_warranty', 'desktop_screen_size', 'pc_cpu',
                                'pc_ram', 'pc_vga', 'pc_drive_capacity', 'elt_origin', 'usage_information', 'classify', 'price']

In [ ]:
name_dict = {
    'Tình trạng': 'elt_condition',
    'Hãng': 'pc_brand',
    'Dòng máy': 'pc_model',
    'Bộ vi xử lý': 'pc_cpu',
    'RAM': 'pc_ram',
    'Ổ cứng': 'pc_drive_capacity',
    'Card màn hình': 'pc_vga',
    'Kích cỡ màn hình': 'desktop_screen_size',
    'Chính sách bảo hành': 'elt_warranty',
    'Xuất xứ': 'elt_origin',
    'Thông tin sử dụng': 'usage_information',
    'Giá': 'price'
}

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
df = df.rename(columns=name_dict)
df.to_csv(csv_file, index=False)
df = pd.read_csv(csv_file)

In [ ]:
df.head(20)

,elt_condition,pc_brand,pc_model,pc_cpu,pc_ram,pc_drive_capacity,usage_information,pc_vga,desktop_screen_size,elt_warranty,elt_origin,classify,price
0,Đã sử dụng (chưa sửa chữa),Asus,VivoBook S Series,Intel Core i7,16 GB,512 GB,In trên bao bì,NVIDIA,15 - 16.9 inch,Còn bảo hành,Đài Loan,laptop,16.900.000 đ
1,Đã sử dụng (chưa sửa chữa),HP,ProBook,Intel Core i5,8 GB,256 GB,In trên bao bì,Onboard,13 - 14.9 inch,7-12 tháng,Đang cập nhật,laptop,4.800.000 đ
2,Đã sử dụng (chưa sửa chữa),Dell,Precision,Intel Core i7,32 GB,512 GB,In trên bao bì,NVIDIA,15 - 16.9 inch,4-6 tháng,Mỹ,laptop,13.000.000 đ
3,Đã sử dụng (chưa sửa chữa),Dell,Vostro,Intel Core i5,8 GB,256 GB,In trên bao bì,Onboard,13 - 14.9 inch,7-12 tháng,Đang cập nhật,laptop,5.700.000 đ
4,Đã sử dụng (chưa sửa chữa),Dell,Latitude,Intel Core i7,8 GB,256 GB,In trên bao bì,Onboard,13 - 14.9 inch,3 tháng,Mỹ,laptop,4.900.000 đ
5,Đã sử dụng (chưa sửa chữa),Dell,Vostro,Intel Core i5,8 GB,500 GB,In trên bao bì,NVIDIA,NaN,Bảo hành hãng,Đang cập nhật,laptop,4.000.000 đ
6,Đã sử dụng (chưa sửa chữa),Apple,MacBook Air,Intel Core i5,8 GB,128 GB,In trên bao bì,Onboard,13 - 14.9 inch,Hết bảo hành,Việt Nam,laptop,9.800.000 đ
7,Đã sử dụng (qua sửa chữa),Dell,Latitude,Intel Core i5,4 GB,512 GB,In trên bao bì,Onboard,13 - 14.9 inch,Hết bảo hành,Trung Quốc,laptop,2.500.000 đ
8,Đã sử dụng (chưa sửa chữa),Apple,MacBook,Intel Core i7,4 GB,128 GB,In trên bao bì,Khác,15 - 16.9 inch,Hết bảo hành,Mỹ,laptop,2.999.999 đ
9,Đã sử dụng (chưa sửa chữa),Sony,S Series,Intel Core i7,8 GB,640 GB,In trên bao bì,Onboard,15 - 16.9 inch,3 tháng,Nhật Bản,laptop,3.900.000 đ


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3994 entries, 0 to 3993
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   elt_condition        3994 non-null   object
 1   pc_brand             3994 non-null   object
 2   pc_model             3994 non-null   object
 3   pc_cpu               3391 non-null   object
 4   pc_ram               3415 non-null   object
 5   pc_drive_capacity    3335 non-null   object
 6   usage_information    3994 non-null   object
 7   pc_vga               2849 non-null   object
 8   desktop_screen_size  3142 non-null   object
 9   elt_warranty         3994 non-null   object
 10  elt_origin           3994 non-null   object
 11  classify             3994 non-null   object
 12  price                3994 non-null   object
dtypes: object(13)
memory usage: 405.8+ KB


# Merge data

In [ ]:
df_desktop = pd.read_csv(folder + '/data_may_ban.csv')
df_laptop = pd.read_csv(folder + '/data_laptop_price_200.csv')
merged_df = pd.concat([df_desktop, df_laptop])
merged_df = merged_df.reset_index(drop=True)

In [ ]:
df_desktop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4010 entries, 0 to 4009
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pc_brand             0 non-null      float64
 1   pc_model             0 non-null      float64
 2   elt_condition        3895 non-null   object 
 3   elt_warranty         3895 non-null   object 
 4   desktop_screen_size  2747 non-null   object 
 5   pc_cpu               3370 non-null   object 
 6   pc_ram               3312 non-null   object 
 7   pc_vga               2790 non-null   object 
 8   pc_drive_capacity    3220 non-null   object 
 9   elt_origin           3895 non-null   object 
 10  usage_information    3895 non-null   object 
 11  classify             4010 non-null   object 
 12  price                3895 non-null   object 
dtypes: float64(2), object(11)
memory usage: 407.4+ KB


In [ ]:
df_laptop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3994 entries, 0 to 3993
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   elt_condition        3994 non-null   object
 1   pc_brand             3994 non-null   object
 2   pc_model             3994 non-null   object
 3   pc_cpu               3391 non-null   object
 4   pc_ram               3415 non-null   object
 5   pc_drive_capacity    3335 non-null   object
 6   usage_information    3994 non-null   object
 7   pc_vga               2849 non-null   object
 8   desktop_screen_size  3142 non-null   object
 9   elt_warranty         3994 non-null   object
 10  elt_origin           3994 non-null   object
 11  classify             3994 non-null   object
 12  price                3994 non-null   object
dtypes: object(13)
memory usage: 405.8+ KB


In [ ]:
df_desktop.head(5)

,pc_brand,pc_model,elt_condition,elt_warranty,desktop_screen_size,pc_cpu,pc_ram,pc_vga,pc_drive_capacity,elt_origin,usage_information,classify,price
0,NaN,NaN,Đã sử dụng (chưa sửa chữa),>12 tháng,>= 21 inch,Intel Xeon,> 32GB,NVIDIA,256 GB,Việt Nam,In trên bao bì,desktop,13.500.000 đ
1,NaN,NaN,Đã sử dụng (chưa sửa chữa),Còn bảo hành,>= 21 inch,Intel Core i5,8 GB,Khác,500 GB,Đang cập nhật,In trên bao bì,desktop,1.800.000 đ
2,NaN,NaN,Đã sử dụng (chưa sửa chữa),Còn bảo hành,>= 21 inch,Intel Xeon,32 GB,NVIDIA,256 GB,Đang cập nhật,In trên bao bì,desktop,8.500.000 đ
3,NaN,NaN,Mới,Còn bảo hành,>= 21 inch,Intel Xeon,> 32GB,NVIDIA,256 GB,Việt Nam,In trên bao bì,desktop,12.500.000 đ
4,NaN,NaN,Đã sử dụng (chưa sửa chữa),3 tháng,Không bán kèm màn hình,Intel Core i3,8 GB,NVIDIA,128 GB,Đang cập nhật,In trên bao bì,desktop,6.300.000 đ


In [ ]:
df_laptop.head(5)

,elt_condition,pc_brand,pc_model,pc_cpu,pc_ram,pc_drive_capacity,usage_information,pc_vga,desktop_screen_size,elt_warranty,elt_origin,classify,price
0,Đã sử dụng (chưa sửa chữa),Asus,VivoBook S Series,Intel Core i7,16 GB,512 GB,In trên bao bì,NVIDIA,15 - 16.9 inch,Còn bảo hành,Đài Loan,laptop,16.900.000 đ
1,Đã sử dụng (chưa sửa chữa),HP,ProBook,Intel Core i5,8 GB,256 GB,In trên bao bì,Onboard,13 - 14.9 inch,7-12 tháng,Đang cập nhật,laptop,4.800.000 đ
2,Đã sử dụng (chưa sửa chữa),Dell,Precision,Intel Core i7,32 GB,512 GB,In trên bao bì,NVIDIA,15 - 16.9 inch,4-6 tháng,Mỹ,laptop,13.000.000 đ
3,Đã sử dụng (chưa sửa chữa),Dell,Vostro,Intel Core i5,8 GB,256 GB,In trên bao bì,Onboard,13 - 14.9 inch,7-12 tháng,Đang cập nhật,laptop,5.700.000 đ
4,Đã sử dụng (chưa sửa chữa),Dell,Latitude,Intel Core i7,8 GB,256 GB,In trên bao bì,Onboard,13 - 14.9 inch,3 tháng,Mỹ,laptop,4.900.000 đ


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8004 entries, 0 to 8003
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   pc_brand             3994 non-null   object
 1   pc_model             3994 non-null   object
 2   elt_condition        7889 non-null   object
 3   elt_warranty         7889 non-null   object
 4   desktop_screen_size  5889 non-null   object
 5   pc_cpu               6761 non-null   object
 6   pc_ram               6727 non-null   object
 7   pc_vga               5639 non-null   object
 8   pc_drive_capacity    6555 non-null   object
 9   elt_origin           7889 non-null   object
 10  usage_information    7889 non-null   object
 11  classify             8004 non-null   object
 12  price                7889 non-null   object
dtypes: object(13)
memory usage: 813.0+ KB


In [ ]:
merged_df.head()

,pc_brand,pc_model,elt_condition,elt_warranty,desktop_screen_size,pc_cpu,pc_ram,pc_vga,pc_drive_capacity,elt_origin,usage_information,classify,price
0,NaN,NaN,Đã sử dụng (chưa sửa chữa),>12 tháng,>= 21 inch,Intel Xeon,> 32GB,NVIDIA,256 GB,Việt Nam,In trên bao bì,desktop,13.500.000 đ
1,NaN,NaN,Đã sử dụng (chưa sửa chữa),Còn bảo hành,>= 21 inch,Intel Core i5,8 GB,Khác,500 GB,Đang cập nhật,In trên bao bì,desktop,1.800.000 đ
2,NaN,NaN,Đã sử dụng (chưa sửa chữa),Còn bảo hành,>= 21 inch,Intel Xeon,32 GB,NVIDIA,256 GB,Đang cập nhật,In trên bao bì,desktop,8.500.000 đ
3,NaN,NaN,Mới,Còn bảo hành,>= 21 inch,Intel Xeon,> 32GB,NVIDIA,256 GB,Việt Nam,In trên bao bì,desktop,12.500.000 đ
4,NaN,NaN,Đã sử dụng (chưa sửa chữa),3 tháng,Không bán kèm màn hình,Intel Core i3,8 GB,NVIDIA,128 GB,Đang cập nhật,In trên bao bì,desktop,6.300.000 đ


In [ ]:
merged_df.to_csv(folder + '/final_data.csv', index=False)